In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)  #to automatically remount make force_remount=True

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [0]:
df=pd.read_csv('/content/gdrive/My Drive/sales.csv')

In [0]:
df.head()
df=df.sort_values(by=['date'])

In [0]:
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,product_pyramid,end_use,solid_novelty,offer_type,color_family
108712,01.01.2013,0,37,15822,28.0,1,Seasonal Fashion,Career,Footwear,New,Beige
87289,01.01.2013,0,54,10650,850.0,1,Basic,Weekend,Print/Pattern,New & Feature,Beige
87267,01.01.2013,0,54,10620,1000.0,1,Basic,Weekend,Print/Pattern,New & Feature,Beige
50287,01.01.2013,0,18,13102,1290.0,1,Accessories,Special Occasion,Accessories,Relist,Pink
87254,01.01.2013,0,54,9736,298.0,1,Basic,Weekend,Print/Pattern,New & Feature,Beige


In [0]:
#The LSTM model below is run only for 1 store, but this can be replicated for each store separately.
shop=0
df_item=df[df['shop_id']==shop]
df_item=df_item[['date','shop_id','item_price']]


In [0]:
df_item.shape

(9857, 3)

In [0]:
df_item.sort_values(by=['date'])


,date,shop_id,item_price
181271,01.02.2013,0,657.0
174945,01.02.2013,0,177.0
174935,01.02.2013,0,1590.0
175131,01.02.2013,0,143.0
175392,01.02.2013,0,1321.0
175389,01.02.2013,0,1128.0
175347,01.02.2013,0,242.0
175153,01.02.2013,0,72.0
175283,01.02.2013,0,137.0
175276,01.02.2013,0,63.0


In [0]:
df_price=pd.DataFrame(columns=['price'],data=df_item['item_price'].values)

In [0]:
df_price.head()

,price
0,657.0
1,657.0
2,231.0
3,190.0
4,190.0


In [0]:
for i in range(1,31):
    df_price["Time_" + str(i)] = df_price.price.shift(i)
df_price.fillna(0.0, inplace=True)


In [0]:
df_price.head(30)

,price,Time_1,Time_2,Time_3,Time_4,Time_5,Time_6,Time_7,Time_8,Time_9,Time_10,Time_11,Time_12,Time_13,Time_14,Time_15,Time_16,Time_17,Time_18,Time_19,Time_20,Time_21,Time_22,Time_23,Time_24,Time_25,Time_26,Time_27,Time_28,Time_29,Time_30
0,657.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,657.0,657.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,231.0,657.0,657.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,190.0,231.0,657.0,657.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,190.0,190.0,231.0,657.0,657.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1136.0,190.0,190.0,231.0,657.0,657.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,242.0,1136.0,190.0,190.0,231.0,657.0,657.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2258.0,242.0,1136.0,190.0,190.0,231.0,657.0,657.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,483.0,2258.0,242.0,1136.0,190.0,190.0,231.0,657.0,657.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,734.0,483.0,2258.0,242.0,1136.0,190.0,190.0,231.0,657.0,657.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
model_lstm = Sequential()
model_lstm.add(LSTM(15, input_shape=(1,30)))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [0]:
scaler = MinMaxScaler(feature_range=(0, 1))
df_price = scaler.fit_transform(df_price)
x=df_price[:9000,1:]
y=df_price[:9000,0]
xval=df_price[9000:,1:]
yval=df_price[9000,0]


In [0]:
x_val_reshape = x.reshape((9000,1,30))
val=xval.reshape((df_price.shape[0]-9000,1,30))

In [0]:
history = model_lstm.fit(x_val_reshape,y,epochs=5, batch_size=1, verbose=2, shuffle=False)


Epoch 1/5
 - 59s - loss: 0.0062 - acc: 3.3333e-04
Epoch 2/5
 - 59s - loss: 0.0062 - acc: 3.3333e-04
Epoch 3/5
 - 59s - loss: 0.0062 - acc: 3.3333e-04
Epoch 4/5
 - 59s - loss: 0.0061 - acc: 3.3333e-04
Epoch 5/5
 - 59s - loss: 0.0061 - acc: 3.3333e-04


In [0]:
y_pre = model_lstm.predict(val)
y_pre.shape

(857, 1)

In [0]:
np.printoptions(precision=4)
print(sum((y_pre-yval)**2)/len(y_pre))  #mean square error


[0.00038185]
